In [153]:
import pandas as pd
import os
import numpy as np

In [159]:
pip install gender_guesser

Note: you may need to restart the kernel to use updated packages.


### Datos de IMDb

In [3]:
df_title_basics = pd.read_csv("data/IMDb/title.basics.tsv", sep="\t", na_values="\\N")
df_title_basics.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_15648\3444689936.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv("data/IMDb/title.basics.tsv", sep="\t", na_values="\\N")


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1.0,Short


In [4]:
df_name_basics = pd.read_csv("data/IMDb/name.basics.tsv", sep="\t", na_values="\\N")
df_name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [166]:
# Vamos a ifentificar si es mujer u hombre: primero miramos la columna primaryProfesion para ver si nos da alguna pista

df_name_basics['is_female'] = df_name_basics['primaryProfession'].apply(
    lambda x: 1 if isinstance(x, str) and 'actress' in x else (
        0 if isinstance(x, str) and 'actor' in x else np.nan
    )
)

# Para los casos en los que aún no sabemos
import gender_guesser.detector as gender
d = gender.Detector()

df_name_basics.loc[df_name_basics['is_female'].isna(), 'is_female'] = (
    df_name_basics[df_name_basics['is_female'].isna()]['primaryName'].apply(
        lambda x: 1 if isinstance(x, str) and d.get_gender(x.split()[0]) in ['female', 'mostly_female']
        else (0 if isinstance(x, str) and d.get_gender(x.split()[0]) in ['male', 'mostly_male'] else np.nan)
    )
)

In [5]:
df_title_crew = pd.read_csv("data/IMDb/title.crew.tsv", sep="\t", na_values="\\N")
df_title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,nm0721526
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


In [6]:
# df_title_principals = pd.read_csv("data/IMDb/title.principals.tsv", sep="\t", na_values="\\N")
# df_title_principals.head()

In [7]:
df_title_ratings = pd.read_csv("data/IMDb/title.ratings.tsv", sep="\t", na_values="\\N")
df_title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2153
1,tt0000002,5.5,292
2,tt0000003,6.5,2188
3,tt0000004,5.3,187
4,tt0000005,6.2,2935


In [8]:
df_title_akas = pd.read_csv("data/IMDb/title.akas.tsv", sep="\t", na_values="\\N")
df_title_akas.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_15648\3311277024.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv("data/IMDb/title.akas.tsv", sep="\t", na_values="\\N")


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,NaN,NaN,original,NaN,1
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita,US,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0


### Datos de la API del test de Bechdel

In [10]:
import requests

# Llamada a la API
url = "https://bechdeltest.com/api/v1/getAllMovies"
response = requests.get(url)

# Convertir a DataFrame
movies = response.json()
df_bechdel = pd.DataFrame(movies)

# Añadir la forma completa del ID de IMDb
df_bechdel["tconst"] = "tt" + df_bechdel["imdbid"].astype(str).str.zfill(7)

# Ver ejemplo
df_bechdel.head()


,title,imdbid,rating,year,id,tconst
0,Grandma Got Run Over by a Reindeer,0267536,3,200,11554,tt0267536
1,Passage de Venus,3155794,0,1874,9602,tt3155794
2,La Rosace Magique,14495706,0,1877,9804,tt14495706
3,Sallie Gardner at a Gallop,2221420,0,1878,9603,tt2221420
4,Le singe musicien,12592084,0,1878,9806,tt12592084


In [11]:
df_bechdel.count()

title     10493
imdbid    10493
rating    10493
year      10493
id        10493
tconst    10493
dtype: int64

In [12]:
df_bechdel[df_bechdel['year'] >= 2010].count()

title     4279
imdbid    4279
rating    4279
year      4279
id        4279
tconst    4279
dtype: int64

### Datos de Kaggle con los nominados a oscars

In [14]:
pip install kagglehub

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("unanimad/the-oscar-award")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\usuario\.cache\kagglehub\datasets\unanimad\the-oscar-award\versions\12


In [16]:
file_path = os.path.join(path, "the_oscar_award.csv")
df_awards = pd.read_csv(file_path)

df_awards.head()

,year_film,year_ceremony,ceremony,category,canon_category,name,film,winner
0,1927,1928,1,ACTOR,ACTOR IN A LEADING ROLE,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,ACTOR IN A LEADING ROLE,Richard Barthelmess,The Patent Leather Kid,False
2,1927,1928,1,ACTOR,ACTOR IN A LEADING ROLE,Emil Jannings,The Last Command,True
3,1927,1928,1,ACTOR,ACTOR IN A LEADING ROLE,Emil Jannings,The Way of All Flesh,True
4,1927,1928,1,ACTRESS,ACTRESS IN A LEADING ROLE,Louise Dresser,A Ship Comes In,False


In [17]:
file_path = os.path.join(path, "full_data.csv")
full_data = pd.read_csv(file_path, delimiter="	")

full_data.head()

,Ceremony,Year,Class,CanonicalCategory,Category,NomId,Film,FilmId,Name,Nominees,NomineeIds,Winner,Detail,Note,Citation,MultifilmNomination
0,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051251,The Noose,tt0019217,Richard Barthelmess,Richard Barthelmess,nm0001932,NaN,Nickie Elkins,NaN,NaN,True
1,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051252,The Patent Leather Kid,tt0018253,Richard Barthelmess,Richard Barthelmess,nm0001932,NaN,The Patent Leather Kid,NaN,NaN,True
2,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051250a,The Last Command,tt0019071,Emil Jannings,Emil Jannings,nm0417837,True,General Dolgorucki [Grand Duke Sergius Alexander],NaN,NaN,True
3,1,1927/28,Acting,ACTOR IN A LEADING ROLE,ACTOR,an0051250b,The Way of All Flesh,tt0019553,Emil Jannings,Emil Jannings,nm0417837,True,August Schilling,NaN,NaN,True
4,1,1927/28,Acting,ACTRESS IN A LEADING ROLE,ACTRESS,an0051255,A Ship Comes In,tt0018389,Louise Dresser,Louise Dresser,nm0237571,NaN,Mrs. Pleznik,NaN,NaN,NaN


In [131]:
full_data['Winner_bool'] = full_data['Winner'].fillna(False).astype(bool)

oscars_grouped = (
    full_data
    .groupby('FilmId')
    .agg(
        total_nominations=('NomId', 'count'),
        total_wins=('Winner_bool', 'sum')
    )
    .reset_index()
)

C:\Users\usuario\AppData\Local\Temp\ipykernel_15648\641047894.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_data['Winner_bool'] = full_data['Winner'].fillna(False).astype(bool)


### Equality Index

In [19]:
equality_index_df = pd.read_csv('data/equaldex_equality_index.csv')

equality_index_df.head()

,Region ID,Name,EI,EI Legal,EI PO,Rank
0,IS,Iceland,94,98,90.0,1
1,NO,Norway,87,87,87.0,2
2,ES,Spain,83,100,66.0,3
3,UY,Uruguay,83,93,73.0,4
4,CL,Chile,81,100,61.0,5


In [20]:
equality_index_df.columns

Index(['Region ID', 'Name', 'EI', 'EI Legal', 'EI PO', 'Rank'], dtype='object')

## Join Data

In [181]:
df_genre_director = (
    df_title_crew
    .assign(
        directors = df_title_crew['directors'].str.split(',')
    ).explode('directors')
    .merge(df_name_basics, 
               how='left', 
               left_on='directors',
               right_on = 'nconst'
              )
    .groupby('tconst')
    .agg(female_director=('is_female', lambda x: 1 if (x == 1).any() else 0))
    .reset_index()
)

In [188]:
df_title_basics_filtered = df_title_basics[
    (df_title_basics['titleType'] == 'movie') & 
    (df_title_basics['isAdult'] < 1) &
    (df_title_basics['startYear'] >= 2010)
]


df_title_info = (
    df_title_basics_filtered
    .merge(df_genre_director, 
           how='left', 
           on='tconst'
          )
    .merge(df_title_ratings, 
           how='left', 
           on='tconst'
          )
    .merge(df_title_akas[df_title_akas['ordering'] == 2][['titleId', 'region']], 
           how='left', 
           left_on='tconst',
           right_on = 'titleId'
          )
    .drop(columns='titleId')
    .merge(equality_index_df, 
           how='left', 
           left_on='region',
           right_on = 'Region ID'
          )
    .drop(columns='Region ID')
    .merge(oscars_grouped, 
           how='left', 
           left_on='tconst',
           right_on = 'FilmId'
          )
    .drop(columns='FilmId')
)
df_title_info[['total_nominations', 'total_wins']] = df_title_info[['total_nominations', 'total_wins']].fillna(0)

In [202]:
df_bechdel_extended =(
    df_bechdel[['rating', 'tconst']]
    .merge(df_title_info[['tconst', 'primaryTitle', 'startYear', 'genres', 'female_director',
       'averageRating', 'region', 'Name', 'EI', 'total_nominations', 'total_wins']], 
           how='left', 
           on='tconst'
          )
)
df_bechdel_extended = df_bechdel_extended[df_bechdel_extended['primaryTitle'].notna()]

In [206]:
df_bechdel_extended.to_csv('movies_bechdel_rating.csv', index=False)